# Symmetric Eigenvalue Decomposition - Algorithms for Tridiagonal Matrices

---

Due to their importance, there is plethora of excellent algorithms for symmetric tridiagonal matrices.

For more details, see 
[I. Slapničar, Symmetric Matrix Eigenvalue Techniques][Hog14] and the references therein.

[Hog14]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 55.1-55.25, CRC Press, Boca Raton, 2014."


## Prerequisites

The reader should be familiar with concepts of eigenvalues and eigenvectors, related perturbation theory, and algorithms. 

 
## Competences 

The reader should be able to apply adequate algorithm to a given symmetric tridiagonal matrix, and to assess its speed and the accuracy of the solution.

## Bisection and inverse iteration

The bisection method is convenient if only part of the spectrum is needed.
If the eigenvectors are needed, as well, they can be efficiently computed by
the inverse iteration method. 

### Facts

$A$ is a real symmetric $n\times n$ matrix and $T$ is a real symmetric
tridiagonal $n\times n$ matrix.
  
1. _(Application of Sylvester's Theorem)_ 
Let $\alpha,\beta\in\mathbb{R}$ with $\alpha<\beta$. The number of
  eigenvalues of $A$ in the interval $[\alpha,\beta)$ is equal to $\nu
  (A- \beta I)-\nu(A-\alpha I)$. By systematically choosing the intervals $[\alpha,\beta)$, the
  bisection method pinpoints each eigenvalue of $A$ to any desired accuracy.

2. The factorization $T-\mu I=LDL^T$, where $D=\mathop{\mathrm{diag}}(d_1,\dots,d_n)$
  and $L$ is the unit lower bidiagonal matrix, is computed as:
\begin{align*}
& d_{1}=T_{11}-\mu, \quad    d_{i}=(T_{ii}-\mu)-
\frac{T_{i,i-1}^2}{d_{i-1}}, \quad i=2,\ldots n, \\
& L_{i+1,i}=\frac{T_{i+1,i}}{d_{i}}, \quad i=1,\ldots,n-1.
\end{align*}
Since the matrices $T$ and $D$ have the same inertia, this recursion
enables an efficient implementation of the bisection method for $T$.

3. The factorization from Fact 2 is essentially Gaussian elimination without
  pivoting. Nevertheless, if $d_i\neq 0$ for all $i$,
  the above recursion is very stable.
  Even when $d_{i-1}=0$ for some $i$, if the IEEE arithmetic is used, the
  computation will continue and the inertia will be computed
  correctly. Namely, in that case, we would have $d_i=-\infty$, $l_{i+1,i}=0$, 
  and $d_{i+1}=t_{i+1.i+1}-\mu$.

4. Computing one eigenvalue of $T$ by using the recursion from Fact 2 and bisection
  requires $O(n)$ operations. The corresponding eigenvector is computed
by inverse iteration. The convergence is very
fast, so the cost of computing each eigenvector is also $O(n)$
operations. Therefore, the overall cost for computing all eigenvalues and
eigenvectors is $O(n^2)$ operations.  

5. Both, bisection and inverse iteration are highly
parallel since each eigenvalue and eigenvector can be computed independently.  

6. If some of the eigenvalues are too close, the corresponding eigenvectors
  computed by inverse iteration may not be sufficiently orthogonal. In this
  case, it is necessary to orthogonalize these eigenvectors  (for example, by the
  modified Gram--Schmidt procedure). If the number of close eigenvalues is too
  large, the overall operation count can increase to $O(n^3)$.

7. The EVD computed by bisection and inverse iteration satisfies
  the error bounds from previous notebook.
  
8. The bisection method for tridiagonal matrices is implemented in the 
LAPACK subroutine [DSTEBZ](http://www.netlib.org/lapack/explore-html/d4/d48/dstebz_8f.html). This routine can compute all eigenvalues
in a given interval or the eigenvalues from $\lambda_l$ to $\lambda_k$, where $l<k$,
and the eigenvalues are ordered from smallest to largest.
Inverse iteration (with reorthogonalization) is implemented in the LAPACK
subroutine DSTEIN. 

In [1]:
a=0

0

In [2]:
b=-a

0

In [3]:
b==a

true

In [4]:
c=1/a

Inf

In [7]:
d=1/(-a)

Inf

In [6]:
c==d

true

In [8]:
n=6
T=SymTridiagonal(rand(n),rand(n-1))

6x6 SymTridiagonal{Float64}:
 0.811349  0.30956   0.0        0.0        0.0       0.0      
 0.30956   0.419992  0.540855   0.0        0.0       0.0      
 0.0       0.540855  0.835802   0.0948444  0.0       0.0      
 0.0       0.0       0.0948444  0.312276   0.654828  0.0      
 0.0       0.0       0.0        0.654828   0.249922  0.903209 
 0.0       0.0       0.0        0.0        0.903209  0.0988848

In [9]:
λ,U=eig(T)

([-0.9105794964646658,-0.03665831334935343,0.23912298092498285,0.8189478581683408,1.2690716846987622,1.3483212890006295],
6x6 Array{Float64,2}:
 -0.00177423  -0.291413    0.0776332    0.871881    0.359603  0.139848
  0.00986917   0.798297   -0.143506     0.0214021   0.531718  0.242586
 -0.0232639   -0.507221    0.00355689  -0.483236    0.628916  0.336334
  0.372082     0.113529    0.795975    -0.0361717  -0.159129  0.434119
 -0.691474     0.0129695  -0.0894357    0.0420035  -0.323602  0.638134
  0.61869     -0.0864241  -0.576014     0.0526869  -0.249772  0.461303)

In [10]:
methods(LAPACK.stebz!)

# 2 methods for generic function "stebz!":
stebz!(range::Char, order::Char, vl::Float64, vu::Float64, il::Integer, iu::Integer, abstol::Real, dv::Array{Float64,1}, ev::Array{Float64,1}) at linalg/lapack.jl:3152
stebz!(range::Char, order::Char, vl::Float32, vu::Float32, il::Integer, iu::Integer, abstol::Real, dv::Array{Float32,1}, ev::Array{Float32,1}) at linalg/lapack.jl:3152

In [11]:
λ1, rest=LAPACK.stebz!('A','E',1.0,1.0,1,1,2*eps(),T.dv,T.ev)

([-0.9105794964646634,-0.03665831334935581,0.2391229809249844,0.8189478581683407,1.2690716846987624,1.3483212890006293],[1,1,1,1,1,1],[6])

In [12]:
λ-λ1

6-element Array{Float64,1}:
 -2.44249e-15
  2.38004e-15
 -1.55431e-15
  1.11022e-16
 -2.22045e-16
  2.22045e-16

In [14]:
U1=LAPACK.stein!(T.dv,T.ev,λ1)

6x6 Array{Float64,2}:
  0.00177423  -0.291413    0.0776332    0.871881    0.359603  0.139848
 -0.00986917   0.798297   -0.143506     0.0214021   0.531718  0.242586
  0.0232639   -0.507221    0.00355689  -0.483236    0.628916  0.336334
 -0.372082     0.113529    0.795975    -0.0361717  -0.159129  0.434119
  0.691474     0.0129695  -0.0894357    0.0420035  -0.323602  0.638134
 -0.61869     -0.0864241  -0.576014     0.0526869  -0.249772  0.461303

In [15]:
# Let us compute just some eigenvalues - from 2nd to 4th
λ2,rest=LAPACK.stebz!('I','E',1.0,1.0,2,4,2*eps(),T.dv,T.ev)

([-0.03665831334935581,0.2391229809249844,0.8189478581683407],[1,1,1],[6])

In [16]:
U2=LAPACK.stein!(T.dv,T.ev,λ2)

6x3 Array{Float64,2}:
 -0.291413    0.0776332    0.871881 
  0.798297   -0.143506     0.0214021
 -0.507221    0.00355689  -0.483236 
  0.113529    0.795975    -0.0361717
  0.0129695  -0.0894357    0.0420035
 -0.0864241  -0.576014     0.0526869

## Divide-and-conquer

This is currently the fastest method for computing the EVD of a real symmetric
tridiagonal matrix $T$. It is based on splitting the given tridiagonal matrix
into two matrices, then computing the EVDs of the smaller matrices and
computing the final EVD from the two EVDs.


$T$ is a real symmetric tridiagonal matrix of order $n$ and
$T=U\Lambda U^T$ is its EVD.

### Facts

1. Let $T$ be partitioned as
$$
T=\begin{bmatrix} T_1 & \alpha_k e_k e_1^T \\
\alpha_k e_1 e_k^T & T_2
\end{bmatrix}.
$$
We assume that $T$ is unreduced, that is, $\alpha_i\neq 0$ for all $i$. Further, we
assume that $\alpha_i>0$ for all $i$, which can be easily be attained by diagonal
similarity with a diagonal matrix of signs.
Let
$$
\hat T_1=T_1- \alpha_k e_k e_k^T,\qquad
\hat T_2=T_2- \alpha_k e_1 e_1^T.
$$
In other words, $\hat T_1$ is equal to $T_1$ except that $T_{kk}$ is replaced by $T_{kk}-\alpha_k$, and 
$\hat T_2$ is equal to $T_2$ except that $T_{k+1,k+1}$ is replaced by $T_{k+1,k+1}-\alpha_k$.
Let $\hat T_i= \hat U_i \hat \Lambda_i \hat U_i^T$, $i=1,2$, be the
respective EVDs and let 
$v=\begin{bmatrix}
\hat U_1^T e_k   \\
\hat U_2^T e_1
\end{bmatrix}
$
($v$ consists of the last column of $\hat U_1^T$ and the
first column of $\hat U_2^T$). Set $\hat U=\hat U_1\oplus \hat U_2$ and $\hat
\Lambda=\hat \Lambda_1 \oplus \hat \Lambda_2$.
Then
$$
T=\begin{bmatrix}\hat U_1 & \\ & \hat U_2 \end{bmatrix}
\left[\begin{bmatrix} \hat \Lambda_1 & \\ & \hat \Lambda_2 \end{bmatrix} + \alpha_k v v^T\right] 
\begin{bmatrix} \hat U_1^T & \\ & \hat U_2^T \end{bmatrix}=
\hat U (\hat \Lambda + \alpha_k v v^T) \hat U^T.
$$
If $\hat \Lambda + \alpha_k v v^T=X\Lambda X^T$
is the EVD of the rank-one modification of the diagonal matrix $\hat \Lambda$,
then $T=U\Lambda U^T$, where $U=\hat U X$  is the EVD of $T$. 
Thus, the original tridiagonal eigenvalue problem
is reduced to two smaller tridiagonal eigenvalue problems and one 
eigenvalue problem for the diagonal-plus-rank-one matrix.

2. If all $\hat \lambda_i$ are different, then the eigenvalues $\lambda_i$
of $\hat \Lambda + \alpha_k v v^T$ are solutions of the so-called secular
equation,
$$
1+e_k \sum_{i=1}^n \frac{v_i^2}{\hat \lambda_i-\lambda}=0.
$$
The eigenvalues can be computed by bisection, or by some faster  zero finder of the Newton type, and they
need to be computed as accurately as possible. The corresponding eigenvectors are 
$$
x_i=(\hat \Lambda-\lambda_i I)^{-1}v.
$$

3. Each $\lambda_i$ and $x_i$ in $O(n)$ operations, respectively, so the overall computational cost for computing
   the EVD of $\hat \Lambda + \alpha_k v v^T$ is $O(n^2)$ operations. 
  
4. The method can be implemented so that the accuracy of the computed EVD is given by the bound from the previous notebook.

5. Tridiagonal Divide-and-conquer method is implemented in the LAPACK subroutine 
[DSTEDC](http://www.netlib.org/lapack/explore-html/d7/d82/dstedc_8f.html). This routine can compute just the
eigenvalues or both, eigenvalues and eigenvectors. 

The file [lapack.jl](https://github.com/JuliaLang/julia/blob/master/base/linalg/lapack.jl) contains wrappers for a selection of LAPACK routines needed in the current Julia `Base`. However, _all_ LAPACK routines are in the compiled library, so  additional wrappers can be easily written. Notice that arrays are passed directly and scalars as passed as pointers. The wrapper for `DSTEDC`, similar to the ones from the file 
`lapack.jl` follows.

In [18]:
# Part of the preamble of lapack.jl
const liblapack = Base.liblapack_name
import Base.blasfunc
# import ..LinAlg: BlasFloat, Char, BlasInt, LAPACKException,
    # DimensionMismatch, SingularException, PosDefException, chkstride1, chksquare
import Base.LinAlg.BlasInt
macro assertargsok() #Handle only negative info codes - use only if positive info code 
    # is useful! 
    :(info[1]<0 && throw(ArgumentError("invalid argument #$(-info[1]) to LAPACK call"))) 
end 
macro lapackerror() #Handle all nonzero info codes 
    :(info[1]>0 ? throw(LAPACKException(info[1])) : @assertargsok ) 
    end 

In [19]:
for (stedc, elty) in
    ((:dstedc_,:Float64),
    (:sstedc_,:Float32))
    @eval begin
        """
        COMPZ is CHARACTER*1
          = 'N':  Compute eigenvalues only.
          = 'I':  Compute eigenvectors of tridiagonal matrix also.
          = 'V':  Compute eigenvectors of original dense symmetric
                  matrix also.  On entry, Z contains the orthogonal
                  matrix used to reduce the original matrix to
                  tridiagonal form.
        """
        function stedc!(compz::Char, dv::Vector{$elty}, ev::Vector{$elty}, Z::Array{$elty})
            n = length(dv)
            ldz=n
            if length(ev) != n - 1
                throw(DimensionMismatch("ev has length $(length(ev)) but needs one less than dv's length, $n)"))
            end
            w = deepcopy(dv)
            u = deepcopy(ev)
            lwork=5*n^2
            work = Array($elty, lwork)
            liwork=6+6*n+5*n*round(Int,ceil(log(n)/log(2)))
            iwork = Array(BlasInt,liwork)
            info = Array(BlasInt,1)
            ccall(($(blasfunc(stedc)), liblapack), Void,
                (Ptr{UInt8}, Ptr{BlasInt}, Ptr{$elty},
                Ptr{$elty}, Ptr{$elty}, Ptr{BlasInt}, Ptr{$elty}, Ptr{BlasInt},
                Ptr{BlasInt}, Ptr{BlasInt}, Ptr{BlasInt}),
                &compz, &n, w,
                u, Z, &ldz, work, &lwork, 
                iwork, &liwork, info) 
                @lapackerror
            w,Z
        end
    end
end

In [20]:
μ,Q=stedc!('I',T.dv,T.ev,eye(n))

([-0.910579496464663,-0.03665831334935599,0.23912298092498444,0.8189478581683404,1.269071684698762,1.3483212890006295],
6x6 Array{Float64,2}:
 -0.00177423  -0.291413   -0.0776332    0.871881    0.359603  -0.139848
  0.00986917   0.798297    0.143506     0.0214021   0.531718  -0.242586
 -0.0232639   -0.507221   -0.00355689  -0.483236    0.628916  -0.336334
  0.372082     0.113529   -0.795975    -0.0361717  -0.159129  -0.434119
 -0.691474     0.0129695   0.0894357    0.0420035  -0.323602  -0.638134
  0.61869     -0.0864241   0.576014     0.0526869  -0.249772  -0.461303)

In [21]:
λ-μ

6-element Array{Float64,1}:
 -2.88658e-15
  2.56045e-15
 -1.58207e-15
  4.44089e-16
  2.22045e-16
  0.0        

In [22]:
Q'*(T*Q)

6x6 Array{Float64,2}:
 -0.910579     -1.27008e-16   4.31717e-16  …  -1.8841e-16   -2.55267e-16
 -1.23653e-16  -0.0366583    -6.67986e-17      9.58373e-17  -1.35889e-16
  4.42894e-16  -7.36171e-17   0.239123         1.00504e-16   2.51523e-16
  8.29837e-17   7.69861e-17  -5.0937e-17       7.3027e-17    1.02729e-16
 -1.64735e-16   8.47351e-17   1.13126e-16      1.26907      -5.9101e-17 
 -2.5403e-16   -9.83581e-17   1.48495e-16  …  -8.46863e-18   1.34832    

In [23]:
# Timings
n=3000
Tbig=SymTridiagonal(rand(n),rand(n-1));

In [24]:
@time eig(Tbig);
@time stedc!('I',Tbig.dv,Tbig.ev,eye(n));

  1.295860 seconds (225 allocations: 138.143 MB, 0.61% gc time)
  0.303881 seconds (24 allocations: 413.545 MB, 14.76% gc time)


## MRRR

The method of Multiple Relatively Robust Representations

The computation of the tridiagonal EVD which satisfies the
error stabdard error bounds such that the eigenvectors are orthogonal to
working precision, all in $O(n^2)$ operations, has been the _holy
grail_ of numerical linear algebra for a long time. 
The method of Multiple Relatively Robust Representations does the job, except in some
exceptional cases. The key idea is to implement inverse iteration more
carefully. The practical algorithm is quite
elaborate and the reader is advised to consider references.

The MRRR method is implemented in the LAPACK subroutine 
[DSTEGR](http://www.netlib.org/lapack/explore-html/d0/d3b/dstegr_8f.html). 
This routine can compute just the
eigenvalues, or both  eigenvalues and eigenvectors.

In [25]:
methods(LAPACK.stegr!)

# 3 methods for generic function "stegr!":
stegr!(jobz::Char, range::Char, dv::Array{Float64,1}, ev::Array{Float64,1}, vl::Real, vu::Real, il::Integer, iu::Integer) at linalg/lapack.jl:3180
stegr!(jobz::Char, range::Char, dv::Array{Float32,1}, ev::Array{Float32,1}, vl::Real, vu::Real, il::Integer, iu::Integer) at linalg/lapack.jl:3180
stegr!(jobz::Char, dv::Array{T,1}, ev::Array{T,1}) at linalg/lapack.jl:3268

In [26]:
LAPACK.stegr!('V',T.dv,T.ev)

([-0.9105794964646658,-0.03665831334935343,0.23912298092498285,0.8189478581683408,1.2690716846987622,1.3483212890006295],
6x6 Array{Float64,2}:
 -0.00177423  -0.291413    0.0776332    0.871881    0.359603  0.139848
  0.00986917   0.798297   -0.143506     0.0214021   0.531718  0.242586
 -0.0232639   -0.507221    0.00355689  -0.483236    0.628916  0.336334
  0.372082     0.113529    0.795975    -0.0361717  -0.159129  0.434119
 -0.691474     0.0129695  -0.0894357    0.0420035  -0.323602  0.638134
  0.61869     -0.0864241  -0.576014     0.0526869  -0.249772  0.461303)

In [27]:
# Timings
@time LAPACK.stegr!('V',Tbig.dv,Tbig.ev);

  1.325125 seconds (71 allocations: 138.087 MB, 3.65% gc time)


In [28]:
n=1500
Tbig=SymTridiagonal(rand(n),rand(n-1));

In [29]:
@time LAPACK.stegr!('V',Tbig.dv,Tbig.ev);

  0.334723 seconds (68 allocations: 34.713 MB, 4.96% gc time)


In [40]:
a=0.0

0.0

In [41]:
b=-a

-0.0

In [42]:
c=1/a

Inf

In [43]:
d=1/b

-Inf

In [44]:
a==b

true

In [45]:
c==d

false